### Initial step: From data we need to create graph of cocuring hashtags.


In [1]:
import pandas as pd
import networkx as nx

In [2]:
videos = pd.read_json("../shared-folder-gald/data/video-creators.json")

In [4]:
videos.head()

,username,video_description,region_code,share_count,hashtag_names,id,like_count,music_id,view_count,voice_to_text,comment_count,create_time,cluster,cluster-label,playlist_id,effect_ids
0,the.animal.holocaust,Replying to @ov10bronco #dominicizrealmyers #v...,US,4,"[vegan, yes, bbq, meat, carnivore, govegan, ve...",7274212644502998314,40,7.274213e+18,1054,"You say, why torture yourself? Because all we ...",181.0,2023-09-02 13:00:37,10,Healthy Cooking,NaN,NaN
1,bakemehealthylove,How to make: 1. Get Your Mix: Grab our Oatmeal...,US,4,"[wafflemix, waffleday, plantbased, strawberryw...",7271102720256314666,65,7.217848e+18,1087,Happy National Waffle Day we're celebrating wi...,0.0,2023-08-25 03:52:14,10,Healthy Cooking,NaN,NaN
2,livinapril7,Looking for a healthy #plantbased #protein pac...,US,0,"[food, healthy, lunch, health, salad, tasty, o...",7272105378920353054,2,6.705026e+18,271,NaN,0.0,2023-08-27 20:43:20,10,Healthy Cooking,NaN,NaN
3,goodvibessocietyofficial,"Pull up to Sprouts, grab those good vibes bott...",US,0,"[drinks, weekendvibes, sprouts, plantbased, su...",7271427012529524011,31,7.271427e+18,372,NaN,2.0,2023-08-26 00:50:41,-1,Outliers,NaN,NaN
4,settonfarms,"At Setton Farms, we are committed to sustainab...",US,1,"[nuts, recipe, farm, farmlife, harvest, pistac...",7273980908565433646,120,7.133309e+18,7031,NaN,2.0,2023-09-01 22:01:08,8,Gardening,NaN,NaN


In [3]:
hashtag_dict = videos.groupby('id')['hashtag_names'].apply(list).to_dict()

hashtag_set = [(video, hashtag) for video, hashtags in hashtag_dict.items() for hashtag in hashtags[0]]

# Create a DataFrame
hashtag_df = pd.DataFrame(hashtag_set, columns=['id', 'hashtag_names'])
hashtag_df



,id,hashtag_names
0,7151417313785335045,autumn
1,7151417313785335045,fall
2,7151417313785335045,gardening
3,7151417313785335045,gardeners
4,7151417313785335045,organicgardening
...,...,...
8856020,7426010179746991391,atlanticforest
8856021,7426085565751184672,nature
8856022,7426085565751184672,construction
8856023,7426085565751184672,hempcrete


In [ ]:
# Sample DataFrame (assuming it's already loaded)
# hashtag_df = pd.read_csv("your_data.csv")  # Load your data if needed

# Count occurrences of each hashtag
hashtag_counts = hashtag_df["hashtag_names"].value_counts()

# Define a threshold (e.g., keep only hashtags that appear at least 100 times)
threshold = 2  
frequent_hashtags = hashtag_counts[hashtag_counts >= threshold]

# Filter the original DataFrame
filtered_df = hashtag_df[hashtag_df["hashtag_names"].isin(frequent_hashtags.index)]

# Display results
print(f"Original unique hashtags: {hashtag_df['hashtag_names'].nunique()}")
print(f"Filtered unique hashtags: {filtered_df['hashtag_names'].nunique()}")

# Save or use the filtered data
# filtered_df.to_csv("filtered_hashtags.csv", index=False)


Original unique hashtags: 673697
Filtered unique hashtags: 286836


In [ ]:
# # Save as CSV
# filtered_df.to_csv("edgelist_filtered.csv", index=False)


### Creating a bipartitre graph for videos-hashtags

In [11]:
# UNFILTERED GRAPH
# G = nx.Graph()
# #for nodes

# video_ids = set(hashtag_df['id'])
# hashtags = set(hashtag_df['hashtag_names'])

# G.add_nodes_from(video_ids, bipartite=0)  # First set (videos)
# G.add_nodes_from(hashtags, bipartite=1)   # Second set (hashtags)

# #for edges

# for _, row in hashtag_df.iterrows():
#     G.add_edge(row['id'], row['hashtag_names'])

#FILTERED GRAPH
G = nx.Graph()
#for nodes

video_ids = set(filtered_df['id'])
hashtags = set(filtered_df['hashtag_names'])

G.add_nodes_from(video_ids, bipartite=0)  # First set (videos)
G.add_nodes_from(hashtags, bipartite=1)   # Second set (hashtags)

#for edges

for _, row in filtered_df.iterrows():
    G.add_edge(row['id'], row['hashtag_names'])


Note to self: create a bayesian graph for the report

### Uniparted network of hashtags

Two hashtags are connected if they appeared in the same video.

The edge weight represents how many times they co-occurred.

In [12]:

# projecting to a unipartite graph of hashtags
HG = nx.bipartite.weighted_projected_graph(G, hashtags, ratio=False)
HG


KeyboardInterrupt: 

In [ ]:
# %%
nx.write_graphml(HG, "filtered_bipartite.graphml")

# %%
import pickle

with open("filtered_bipartite.pkl", "wb") as f:
    pickle.dump(HG, f)

# %%
import csv

edges_with_weights = HG.edges(data=True)

# Open a CSV file to write
with open("edgelist_unipartite_filtered.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Source", "Target", "Weight"])  # Column headers
    
    # Write the edges and weights
    for u, v, weight in edges_with_weights:
        writer.writerow([u, v, weight['weight']])
